In [7]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import iris.analysis.cartography
import iris.plot as iplt
import iris.quickplot as qplt
import numpy as np
import matplotlib.ticker as mticker 
from numpy import *
from matplotlib.pyplot import *
from iris.analysis import *
from iris.coord_categorisation import * 
from iris.analysis.cartography import * 
from scipy.stats import *
from scipy.stats.mstats import *
from iris import *

fname = '/Users/alicebrindley/OneDrive - University of Exeter/siconc_SImon_GFDL-CM4_ssp585_r1i1p1f1_gr_201501-210012.nc'
cube = iris.load(fname)
print(cube)

siconc=cube[2] 
print(siconc)

0: longitude / (degrees)               (latitude: 180; longitude: 360)
1: latitude / (degrees)                (latitude: 180; longitude: 360)
2: sea_ice_area_fraction / (%)         (time: 1032; latitude: 180; longitude: 360)
sea_ice_area_fraction / (%)         (time: 1032; latitude: 180; longitude: 360)
    Dimension coordinates:
        time                             x               -               -
        latitude                         -               x               -
        longitude                        -               -               x
    Cell methods:
        mean where sea              area
        mean                        time
    Attributes:
        Conventions                 CF-1.7 CMIP-6.0 UGRID-1.0
        activity_id                 ScenarioMIP
        branch_method               standard
        branch_time_in_child        0.0
        branch_time_in_parent       60225.0
        comment                     Model data on the 1x1 grid includes values in all ce

/Users/alicebrindley/opt/miniconda3/lib/python3.9/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacello', referenced by netCDF variable 'siconc'
  warnings.warn(


In [8]:

#extract Jan and Dec to average the sea ice concentration across these mon
add_month_number(siconc, 'time', name='month_number2')
cube2 = siconc[np.where((siconc.coord('month_number2').points == 12))]
#then to average this by each year, so that you have the December-Jan for
add_season_year(cube2, 'time', name='season_year')
#then average by the season year:
cube2.aggregated_by(['season_year'], iris.analysis.MEAN)
#cube2 is therefore the data per season


<iris 'Cube' of sea_ice_area_fraction / (%) (time: 86; latitude: 180; longitude: 360)>

In [ ]:
#identify the Ross Sea
#start of the loop
for i in range (0,2050-2015):
    cube3=cube2[i,:,:]
    west = -230
    east = -110
    south = -85
    north = -60
    temporary_cube = cube3.intersection(longitude = (west, east)) 
    regional_cube = temporary_cube.intersection(latitude = (south, north))
    ax1 = plt.subplot(111,projection=ccrs.PlateCarree(central_longitude=np.round(west + (east - west)/2.0)))
    #my_plot = iplt.contourf(regional_cube)
    #plt.gca().gridlines(draw_labels=True)
    #plt.show()


# most northerly row of cells and have a criteria like the average sea ice
# row is > 80% (might need some testing/iterating to find the right number
#extract the latitude I want to work with
    north_edge = regional_cube.extract(iris.Constraint(latitude=-70))
    #average across this latitude
    nE_mean = north_edge.collapsed('longitude', iris.analysis.MEAN)
    #print(nE_mean.data)
    
    if nE_mean.data >=70:
        #define siconc for polynya
        polynya_threshold=regional_cube.data
        polynya = np.where((polynya_threshold.data <=30) & (nE_mean.data >=70))
        #print(polynya)
        polynya = polynya_threshold.data
        plt.pcolormesh(polynya_threshold.data,vmin=0,vmax=100)
        #plt.show()


        #define a variable to hold the threshold number you are interested in
        polynya_threshold_value = 30.0
        #Take a copy of the seaice data that we are going to use to make a binary 'mask' showing presence of absense of polynyas as 1s and 0s
        polynya_location = polynya_threshold.data.copy()
        #set that land data in this mask to nan (Not a number) so we don;t get confused (these will show up white in a typical plot)
        polynya_location[np.where(polynya_location >= 1.0e9)] = np.nan
        #make water that has more ice than our polynia condition equal to zero.
        polynya_location[np.where((polynya_location <= 100.0) & (polynya_location > polynya_threshold_value))] = 0.0
        #make water that has less ice than our polynia condition equal to zero to make where the polynya is
        polynya_location[np.where((polynya_location > 0.0) & (polynya_location <= polynya_threshold_value))] = 1.0
        #plot this data
        plt.pcolormesh(polynya_location,vmin=0,vmax=1)
        #plt.show()
        #Note in this 1st situation, we have not tested to make sure the northern edge has >30% seaice and it has not, so it is eroniously identify
    else:
        print(cube3.coord('time'))

DimCoord([2017-12-16 12:00:00], bounds=[[2017-12-01 00:00:00, 2018-01-01 00:00:00]], standard_name='time', calendar='365_day', long_name='time', var_name='time', attributes={'calendar_type': 'noleap', 'description': 'Temporal mean'})


In [42]:
print(cube3)

sea_ice_area_fraction / (%)         (latitude: 180; longitude: 360)
    Dimension coordinates:
        latitude                             x               -
        longitude                            -               x
    Scalar coordinates:
        month_number2               12
        season_year                 2018
        time                        2017-12-16 12:00:00, bound=(2017-12-01 00:00:00, 2018-01-01 00:00:00)
    Cell methods:
        mean where sea              area
        mean                        time
    Attributes:
        Conventions                 CF-1.7 CMIP-6.0 UGRID-1.0
        activity_id                 ScenarioMIP
        branch_method               standard
        branch_time_in_child        0.0
        branch_time_in_parent       60225.0
        comment                     Model data on the 1x1 grid includes values in all cells for which any ocean...
        contact                     gfdl.climate.model.info@noaa.gov
        creation_date         

In [10]:
#define a variable to hold the threshold number you are interested in

for i in range (0,2050-2015):
    cube3=cube2[i,:,:]
    polynya_threshold_value = 30.0

    #Take a copy of the seaice data that we are going to use to make a binary 'mask' showing presence of absense of polynyas as 1s and 0s
    polynya_location = polynya_threshold.data.copy()

    #set that land data in this mask to nan (Not a number) so we don;t get confused (these will show up white in a typical plot)
    polynya_location[np.where(polynya_location >= 1.0e9)] = np.nan

    #make water that has more ice than our polynia condition equal to zero.
    polynya_location[np.where((polynya_location <= 100.0) & (polynya_location > polynya_threshold_value))] = 0.0

    #make water that has less ice than our polynia condition equal to zero to make where the polynya is
    polynya_location[np.where((polynya_location > 0.0) & (polynya_location <= polynya_threshold_value))] = 1.0

    #plot this data
    plt.pcolormesh(polynya_location,vmin=0,vmax=1)

#Note in this 1st situation, we have not tested to make sure the northern edge has >30% seaice and it has not, so it is eroniously identify

NameError: name 'polynya_threshold' is not defined

In [55]:
print(siconc.date)

AttributeError: 'Cube' object has no attribute 'date'